This is my notebook for the week 3 assignment in the Data Science Capstone class



In [11]:
## Import a bunch of libraries that we needed in the labs and such

import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Libraries imported.


In [12]:
##Input foursquare data

CLIENT_ID = 'OVWKIVZTKAQK3GTX2PG0XO4ESUCBBYEYKT4Q4UW40SO5FI33' # your Foursquare ID
CLIENT_SECRET = 'CWHXW2D20OMRC5H115UQL1CEOBYTCHECZV5PPIQJLTJDJINO' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30

In [13]:
##Install beautifulsoup4 so we can scrape web data
!conda install -c conda-forge beautifulsoup4 --yes

Solving environment: done

# All requested packages already installed.



In [14]:
##Install a html parser
!conda install -c conda-forge lxml --yes

Solving environment: done

# All requested packages already installed.



In [15]:
##Import Beautiful Soup
from bs4 import BeautifulSoup

In [16]:
##Get the source code from the website using the Requests library
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [17]:
##Parse the html code
soup = BeautifulSoup(source, 'lxml')
#print(soup.prettify())    #I'm commenting this out because it's obnoxiously long

In [18]:
##Find the table
table = soup.find('table')
group = table.tbody.find_all('tr')

print(len(group))
##Get the table headings. These will be needed when we get our dictionary and need to convert to dataframe.
headings = group[0].text.strip()
cols = headings.split('\n')
print(cols)

290
['Postcode', 'Borough', 'Neighbourhood']


In [19]:
## Loop through all of the groupings to get a list of each zip code, its borough, and neighborhood
## This will allow us to make our dictionary, and then our dataframe
count = 0
post_code = []
borough = []
neighborhood = []

for group in group:
    if count == 0:
        pass
    else:
        value = group.text.strip() #This will split each entry into a list
        value_list = value.split('\n')
        #This if statement makes the neighborhood value equal to the borough value if the neighborhood value isn't present.
        if value_list[2] == 'Not assigned':
            value_list[2] = value_list[1]
        post_code.append(value_list[0])
        borough.append(value_list[1])
        neighborhood.append(value_list[2])
        
    count = count+1

In [20]:
#Create the dictionary
dict = {cols[0]:post_code, cols[1]:borough, cols[2]:neighborhood}


In [21]:
#Convert to dataframe
df = pd.DataFrame(dict)
#print(df)

In [22]:
#Remove the boroughs that are not assigned, per the instructions.
df = df[df.Borough != 'Not assigned']
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [23]:
## We need to merge rows that have the same postcode. This is the end of the first part.
foo = lambda a: ", ".join(a) 
foo2 = lambda a: ",".join(a).split(',')[0]

df_new = df.groupby(by='Postcode').agg({'Borough':foo2,'Neighbourhood': foo}).reset_index()



df_new.head(20)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


In [24]:
df_new.shape

(103, 3)

In [25]:
import pandas as pd

In [32]:
#Pull the Latitude and Longitude from the csv file. The geocoder thing wasn't working.
df_read = pd.read_csv('http://cocl.us/Geospatial_data')

df_latlong = df_read[['Latitude','Longitude']].copy()
df_latlong.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


In [35]:
#Combine the dataframe with the neighborhoods and stuff, and the latlong dataframe.
frame = [df_new, df_latlong]
df_total = pd.concat(frame, axis = 1, join = 'inner')
df_total.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848
